In [1]:
import requests
from requests.exceptions import HTTPError
import re
from bs4 import BeautifulSoup
import os
import time
#from multiprocessing.pool import ThreadPool
import concurrent.futures

In [2]:
r = requests.get('https://www.gov.br/receitafederal/pt-br/assuntos/orientacao-tributaria/cadastros/consultas/dados-publicos-cnpj')

In [3]:
def download_html(url, timeout=10, numero_tentativas=2):
    print("Realizando o download da página:", url)
    try:
        req = requests.get(url, timeout=timeout)
        if req.status_code != 200:
            if numero_tentativas > 0:
                print("Não foi possível realizar o download. Erro:", req.status_code)
                print("\nRealizando nova tentativa:")
                return download_html(url, numero_tentativas - 1)
            else:
                print("Número de tentativas excedidas. Erro: {}".format(req.status_code))
                html = None
                return html
        html = req.text
        return html
    except ConnectionError as e:
        print("Erro no download:", e)
        html = None

In [4]:
html = download_html('https://www.gov.br/receitafederal/pt-br/assuntos/orientacao-tributaria/cadastros/consultas/dados-publicos-cnpj')

Realizando o download da página: https://www.gov.br/receitafederal/pt-br/assuntos/orientacao-tributaria/cadastros/consultas/dados-publicos-cnpj


In [5]:
bs = BeautifulSoup(html, "html.parser")

In [6]:
lista_de_links = bs.body.find_all("a", class_="external-link")

In [7]:
lista_de_links

[<a class="external-link" data-tippreview-enabled="true" data-tippreview-image="" data-tippreview-title="" href="http://200.152.38.155/CNPJ/K3241.K03200Y0.D11113.EMPRECSV.zip" target="_self" title="">Dados Abertos CNPJ EMPRESA 01</a>,
 <a class="external-link" data-tippreview-enabled="true" data-tippreview-image="" data-tippreview-title="" href="http://200.152.38.155/CNPJ/K3241.K03200Y1.D11113.EMPRECSV.zip" target="_self" title=""><span>Dados Abertos CNPJ</span> EMPRESA 02</a>,
 <a class="external-link" data-tippreview-enabled="true" data-tippreview-image="" data-tippreview-title="" href="http://200.152.38.155/CNPJ/K3241.K03200Y2.D11113.EMPRECSV.zip" target="_self" title=""><span>Dados Abertos CNPJ</span> EMPRESA 03 </a>,
 <a class="external-link" data-tippreview-enabled="true" data-tippreview-image="" data-tippreview-title="" href="http://200.152.38.155/CNPJ/K3241.K03200Y3.D11113.EMPRECSV.zip" target="_self" title=""><span>Dados Abertos CNPJ</span> EMPRESA 04</a>,
 <a class="external-

In [8]:
for item in lista_de_links:
    print("-->", item.text)

--> Dados Abertos CNPJ EMPRESA 01
--> Dados Abertos CNPJ EMPRESA 02
--> Dados Abertos CNPJ EMPRESA 03 
--> Dados Abertos CNPJ EMPRESA 04
--> Dados Abertos CNPJ EMPRESA 05
--> Dados Abertos CNPJ EMPRESA 06
--> Dados Abertos CNPJ EMPRESA 07 
--> Dados Abertos CNPJ EMPRESA 08
--> Dados Abertos CNPJ EMPRESA 09
--> Dados Abertos CNPJ EMPRESA 10
--> Dados Abertos CNPJ ESTABELECIMENTO 01
--> Dados Abertos CNPJ ESTABELECIMENTO 02
--> Dados Abertos CNPJ ESTABELECIMENTO 03
--> Dados Abertos CNPJ ESTABELECIMENTO 04
--> Dados Abertos CNPJ ESTABELECIMENTO 05
--> Dados Abertos CNPJ ESTABELECIMENTO 06
--> Dados Abertos CNPJ ESTABELECIMENTO 07
--> Dados Abertos CNPJ ESTABELECIMENTO 08
--> Dados Abertos CNPJ ESTABELECIMENTO 09
--> Dados Abertos CNPJ ESTABELECIMENTO 10
--> Dados Abertos CNPJ SÓCIO 01
--> Dados Abertos CNPJ SÓCIO 02
--> Dados Abertos CNPJ SÓCIO 03
--> Dados Abertos CNPJ SÓCIO 04
--> Dados Abertos CNPJ SÓCIO 05
--> Dados Abertos CNPJ SÓCIO 06
--> Dados Abertos CNPJ SÓCIO 07
--> Dados Aber

In [9]:
lista = []
for link in lista_de_links:
    lista.append(link.get("href"))
    print(link.get("href"))

http://200.152.38.155/CNPJ/K3241.K03200Y0.D11113.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y1.D11113.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y2.D11113.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y3.D11113.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y4.D11113.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y5.D11113.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y6.D11113.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y7.D11113.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y8.D11113.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y9.D11113.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y0.D11113.ESTABELE.zip
http://200.152.38.155/CNPJ/K3241.K03200Y1.D11113.ESTABELE.zip
http://200.152.38.155/CNPJ/K3241.K03200Y2.D11113.ESTABELE.zip
http://200.152.38.155/CNPJ/K3241.K03200Y3.D11113.ESTABELE.zip
http://200.152.38.155/CNPJ/K3241.K03200Y4.D11113.ESTABELE.zip
http://200.152.38.155/CNPJ/K3241.K03200Y5.D11113.ESTABELE.zip
http://2

In [10]:
lista[20:22]

['http://http//200.152.38.155/CNPJ/K3241.K03200Y0.D11113.SOCIOCSV.zip',
 'http://http//200.152.38.155/CNPJ/K3241.K03200Y1.D11113.SOCIOCSV.zip']

In [11]:
lista[20] = 'http://200.152.38.155/CNPJ/K3241.K03200Y0.D11113.SOCIOCSV.zip'

In [12]:
lista[21] = 'http://200.152.38.155/CNPJ/K3241.K03200Y1.D11113.SOCIOCSV.zip'

In [13]:
lista

['http://200.152.38.155/CNPJ/K3241.K03200Y0.D11113.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y1.D11113.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y2.D11113.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y3.D11113.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y4.D11113.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y5.D11113.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y6.D11113.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y7.D11113.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y8.D11113.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y9.D11113.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y0.D11113.ESTABELE.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y1.D11113.ESTABELE.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y2.D11113.ESTABELE.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y3.D11113.ESTABELE.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y4.D11113.ESTABELE.zip',
 'http://2

In [14]:
total = len(lista)

In [15]:
total

37

In [16]:
def download_url(link, chunk_size=1024):
    path=r'D:\Arquivos CNPJ'
    file = link.split('/')[-1]
    save_path = path + '\\' + file
    r = requests.get(link, stream=True)
    
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)
            
    return link

In [17]:
print('Iniciando download de {} arquivos:'.format(total))
threaded_start = time.time()
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for url in lista:
        futures.append(executor.submit(download_url, link=url))
    for future in concurrent.futures.as_completed(futures):
        total = total - 1
        print('Faltam {} arquivos. Arquivo baixado do link: {}'.format(total, future.result()))        
print("Threaded time:", time.time() - threaded_start)

Iniciando download de 37 arquivos:
Faltam 36 arquivos. Arquivo baixado do link: http://200.152.38.155/CNPJ/K3241.K03200Y1.D11113.EMPRECSV.zip
Faltam 35 arquivos. Arquivo baixado do link: http://200.152.38.155/CNPJ/K3241.K03200Y2.D11113.EMPRECSV.zip
Faltam 34 arquivos. Arquivo baixado do link: http://200.152.38.155/CNPJ/K3241.K03200Y4.D11113.EMPRECSV.zip
Faltam 33 arquivos. Arquivo baixado do link: http://200.152.38.155/CNPJ/K3241.K03200Y3.D11113.EMPRECSV.zip
Faltam 32 arquivos. Arquivo baixado do link: http://200.152.38.155/CNPJ/K3241.K03200Y6.D11113.EMPRECSV.zip
Faltam 31 arquivos. Arquivo baixado do link: http://200.152.38.155/CNPJ/K3241.K03200Y7.D11113.EMPRECSV.zip
Faltam 30 arquivos. Arquivo baixado do link: http://200.152.38.155/CNPJ/K3241.K03200Y5.D11113.EMPRECSV.zip
Faltam 29 arquivos. Arquivo baixado do link: http://200.152.38.155/CNPJ/K3241.K03200Y8.D11113.EMPRECSV.zip
Faltam 28 arquivos. Arquivo baixado do link: http://200.152.38.155/CNPJ/K3241.K03200Y0.D11113.EMPRECSV.zip
Fa

In [ ]:
17:07